# PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    stream=sys.stdout,
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"
c.settings.dirs.input_minimal = "../../datasets/inputs/"
c.settings.dirs.preprocess = "../../inputs/preprocess/"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-02-23 21:02:58,461 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: 1,2,6,7
  dirs:
    working: ..
    input: ../../inputs/
    input_minimal: ../../datasets/inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ../../inputs/preprocess/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 440
  n_class: 1
  preprocess: null
  pca_n_components: 50
  n_fold: 5
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_group
  group_name: investment_id
  ti

In [9]:
import torch

In [10]:
import src.utils as utils

In [11]:
device = utils.gpu_settings(c)

2022-02-23 21:03:02,342 [INFO] [utils] CUDA_VISIBLE_DEVICES: 1,2,6,7
2022-02-23 21:03:02,407 [INFO] [utils] torch device: cuda, device count: 4


## Preprocess

In [12]:
from src.load_data import InputData

2022-02-23 20:51:16,128 [INFO] [loader] Loading faiss with AVX2 support.
2022-02-23 20:51:16,248 [INFO] [loader] Successfully loaded faiss with AVX2 support.


In [ ]:
input = InputData(c)

2022-02-23 20:51:16,284 [INFO] [load_data] Load feather file. path: ../../inputs/train.f


In [ ]:
input.train.describe()

## Fold

In [ ]:
input.train.fillna({"time_fold": c.params.n_fold}, inplace=True)

In [ ]:
input.train

In [ ]:
input.train["group_fold"].value_counts()

In [ ]:
input.train["time_fold"].value_counts()

In [ ]:
input.train[["group_fold", "time_fold"]].value_counts()

In [ ]:
from src.make_fold import train_test_split

In [ ]:
train_folds, valid_folds = train_test_split(c, input.train, 1)

In [ ]:
len(train_folds)

In [ ]:
len(valid_folds)

## Model

In [12]:
from src.make_model import make_model

In [13]:
# c.params.model = "ump_1"

In [14]:
model = make_model(c, device)

In [15]:
model

DataParallel(
  (module): MLPModel(
    (bn_1): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc_1): Linear(in_features=1200, out_features=1200, bias=True)
    (fc_2): Linear(in_features=1200, out_features=600, bias=True)
    (fc_3): Linear(in_features=600, out_features=300, bias=True)
    (fc_4): Linear(in_features=300, out_features=150, bias=True)
    (head): Linear(in_features=150, out_features=1, bias=True)
  )
)

In [16]:
input_ = torch.randn(c.params.batch_size, c.params.model_input).to(device)
# input_ = torch.randn(c.params.batch_size, c.params.model_window, c.params.model_input).to(device)

# num_gpu = len(c.settings.gpus.split(","))
# hidden = torch.zeros(1, c.params.batch_size // num_gpu, 300).to(device)
# cell = torch.zeros(1, c.params.batch_size // num_gpu, 300).to(device)
# hidden_cell = (hidden, cell)

In [17]:
input_.size()

torch.Size([640, 1200])

In [18]:
res = model(input_)
# res, hidden_cell = model(input_, hidden_cell)
# res, hidden_cell = model(input_)

In [19]:
res.shape

torch.Size([640])

In [20]:
res

tensor([-0.0347, -0.0347, -0.0326, -0.0423, -0.0451, -0.0416, -0.0492, -0.0407,
        -0.0433, -0.0388, -0.0417, -0.0433, -0.0372, -0.0466, -0.0405, -0.0416,
        -0.0375, -0.0389, -0.0367, -0.0436, -0.0344, -0.0459, -0.0380, -0.0429,
        -0.0372, -0.0475, -0.0394, -0.0402, -0.0399, -0.0335, -0.0376, -0.0386,
        -0.0421, -0.0390, -0.0384, -0.0422, -0.0467, -0.0380, -0.0401, -0.0322,
        -0.0332, -0.0365, -0.0313, -0.0392, -0.0343, -0.0355, -0.0362, -0.0347,
        -0.0318, -0.0419, -0.0421, -0.0349, -0.0415, -0.0384, -0.0375, -0.0421,
        -0.0451, -0.0361, -0.0482, -0.0298, -0.0397, -0.0409, -0.0407, -0.0453,
        -0.0366, -0.0469, -0.0464, -0.0379, -0.0443, -0.0392, -0.0406, -0.0363,
        -0.0354, -0.0349, -0.0447, -0.0414, -0.0392, -0.0469, -0.0400, -0.0427,
        -0.0359, -0.0517, -0.0354, -0.0406, -0.0355, -0.0437, -0.0428, -0.0413,
        -0.0454, -0.0409, -0.0429, -0.0475, -0.0498, -0.0372, -0.0396, -0.0471,
        -0.0354, -0.0412, -0.0407, -0.03